#Obteniendo el DataSet

In [1]:
from google.colab import files
uploaded = files.upload()

Saving ObesityDataSet_raw_and_data_sinthetic.csv to ObesityDataSet_raw_and_data_sinthetic.csv


DataSet obtenido de: [Obesity based on eating habits & physical cond](https://www.kaggle.com/ankurbajaj9/obesity-levels?select=ObesityDataSet_raw_and_data_sinthetic.csv)

In [2]:
import pandas as pd
import io
data = pd.read_csv(io.BytesIO(uploaded['ObesityDataSet_raw_and_data_sinthetic.csv']))

In [3]:
data

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


# Preprocesamiento
Frequent consumption of high caloric food (FAVC)
Frequency of consumption of vegetables(FCVC)
Number of main meals (NCP)
Consumption of food between meals (CAEC)
Consumption of water daily (CH2O)
Calories consumption monitoring (SCC)
Physical activity frequency (FAF)
Time using technology devices (TUE)
Consumption of alcohol (CALC)
Transportation used (MTRANS)

Variables necesarias: el sexo, edad, estatura, alimentación y realización de actividades físicas.

In [86]:
data_pre = data.drop(['family_history_with_overweight','TUE', 'NCP','CAEC','SCC','MTRANS'], axis = 1)

In [6]:
#data_pre.loc[data_pre['SMOKE']=='yes']
#data_pre.drop(['SMOKE'],axis=0, inplace=True)

**Siendo:'Normal_Weight' = 0 'Overweight_Level_I' = 1 'Overweight_Level_II' = 2 'Obesity_Type_I' = 3 'Obesity_Type_II' = 4 'Obesity_Type_III' = 5 'Insufficient_Weight' = 6**



In [58]:
data_pre.replace(['Normal_Weight','Overweight_Level_I','Overweight_Level_II','Obesity_Type_I','Obesity_Type_II','Obesity_Type_III','Insufficient_Weight'],[0,1,2,3,4,5,6], inplace=True)

**Cambio de genero de variable categórica a entera**


In [59]:
data_pre.replace(["Female","Male"],[0,1], inplace=True)

**Cambio de variable categorica a entera para la frecuencia de consumo de comida rica en calorias y habito de fumar.**

In [60]:
data_pre.replace(["no","yes"],[0,1], inplace=True)

In [61]:
data_pre.rename(columns={'NObeyesdad':'Target'}, inplace=True)

**Cambio de variable categorica a entera para el consumo de alcohol.**

In [62]:
data_pre.replace(["Sometimes","Frequently","Always"],[0,1,2], inplace=True)

In [66]:
for i in range(1,10):
  data_pre.iloc[:,i] = round(data_pre.iloc[:,i])

In [67]:
data_pre

,Gender,Age,Height,Weight,FAVC,FCVC,SMOKE,CH2O,FAF,CALC,Target
0,0,21.0,2.0,64.0,0,2.0,0,2.0,0.0,0,0
1,0,21.0,2.0,56.0,0,3.0,1,3.0,3.0,0,0
2,1,23.0,2.0,77.0,0,2.0,0,2.0,2.0,1,0
3,1,27.0,2.0,87.0,0,3.0,0,2.0,2.0,1,1
4,1,22.0,2.0,90.0,0,2.0,0,2.0,0.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...
2106,0,21.0,2.0,131.0,1,3.0,0,2.0,2.0,0,5
2107,0,22.0,2.0,134.0,1,3.0,0,2.0,1.0,0,5
2108,0,23.0,2.0,134.0,1,3.0,0,2.0,1.0,0,5
2109,0,24.0,2.0,133.0,1,3.0,0,3.0,1.0,0,5


#Construcción del módelo de predicción con regresión logística

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import numpy as np

In [68]:
x = data_pre.iloc[:,0:10]
y = data_pre.loc[:,'Target']

In [69]:
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size = 0.8, test_size = 0.2,
                                                    random_state=88)

In [70]:
LG_model = LogisticRegression()

In [71]:
LG_model.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [72]:
LG_prediction = LG_model.predict(x_test)

In [75]:
accuracy_score(y_test,LG_prediction)

0.6146572104018913

In [ ]:
LG_cross_val = cross_val_score(LG_model,x,y,cv=15)
LG_cross_val

In [77]:
sample = np.array([[1,45,1.6,113,0,1.3,1,3,1,1]])
LG_model.predict(sample)

array([2])

#Construcción del módelo de predicción con KNN Classifier

In [101]:
from sklearn.neighbors import KNeighborsClassifier

In [102]:
kNN = KNeighborsClassifier(n_neighbors=6, metric='minkowski',p=1)
kNN.fit(x,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=1,
                     weights='uniform')

In [103]:
kNN_Predict = kNN.predict(x_test)

In [104]:
accuracy_score(y_test,kNN_Predict)

0.9101654846335697

In [105]:
kNN_cross_val = cross_val_score(kNN,x,y,cv=25)
kNN_cross_val

array([0.78823529, 0.87058824, 0.83529412, 0.87058824, 0.92941176,
       0.83529412, 0.91764706, 0.85882353, 0.74117647, 0.87058824,
       0.88235294, 0.86904762, 0.9047619 , 0.8452381 , 0.85714286,
       0.78571429, 0.86904762, 0.88095238, 0.88095238, 0.88095238,
       0.88095238, 0.91666667, 0.86904762, 0.8452381 , 0.88095238])

In [106]:
kNN.predict(sample)

array([4])

In [119]:
datos = ("genero (Mujer: 0 / Hombre: 1): ", 
         "edad",
         "altura: ",
         "peso",
         "si hay consumo de comidas ricas en calorias (no :0 / yes :1)",
         "si hay consumo de vegetales (no :0 / yes :1)",
         "si fuma constantemente (no :0 / yes :1)",
         "consumo de agua",
         "frecuencia de actividad física",
         "si hay consumo de alcohol (aveces :0/ frecuentemente :1/ siempre :2)"
         )

In [120]:
def ingreso_datos():
  a = []
  for i in range(len(datos)):
    d = float(input(print("Ingrese "+datos[i])))
    a.append(d)
    array = [a]
  return array

#Ejecución de pruebas
Recuerde que el resultado del array, indicará la predicción del sujeto en base a sus habitos y condiciones físicas.

1.   'Peso normal' = 0
2.   'Sobrepeso nivel 1' = 1
3.   'Sobrepeso nivel 2' = 2
4.   'Obesidad tipo 1' = 3 
5.   'Obesidad tipo 2' = 4 
6.   'Obesidad tipo 3' = 5 
7.   'Insuficiencia de peso' = 6

In [121]:
kNN.predict(ingreso_datos())

Ingrese genero (Mujer: 0 / Hombre: 1): 
0
Ingrese edad
23
Ingrese altura: 
1.50
Ingrese peso
50
Ingrese si hay consumo de comidas ricas en calorias (no :0 / yes :1)
0
Ingrese si hay consumo de vegetales (no :0 / yes :1)
1
Ingrese si fuma constantemente (no :0 / yes :1)
0
Ingrese consumo de agua
2
Ingrese frecuencia de actividad física
3
Ingrese si hay consumo de alcohol (aveces :0/ frecuentemente :1/ siempre :2)
0


array([6])